<a href="https://colab.research.google.com/github/naveenbharathi-it21/Vehicle-dent-and-Scratch-Detection-using-Detectron/blob/main/PredictApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install detectron 2
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-8gcrm_a8
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-8gcrm_a8
  Resolved https://github.com/facebookresearch/detectron2.git to commit 57bdb21249d5418c130d54e2ebdc94dda7a4c01a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6112223 sha256=6c51cdddb232719f4c1e8310490def45

In [ ]:
import torch
import matplotlib

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import skimage.io as io

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Set base params
plt.rcParams["figure.figsize"] = [16,9]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok
!pip install flask
!ngrok authtoken 2Rujb6PN8eLInx365stBbStd7t5_3SQDSW4mctW7TfxFEusDp


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask,request, render_template, jsonify
from flask_ngrok import run_with_ngrok
import base64

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch
import os, json, cv2, random
import skimage.io as io
from detectron2.utils.logger import setup_logger
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer, ColorMode

# Load the model and configure the predictor
cfg_file = "/content/drive/MyDrive/output/config.yaml"
weight_file = "/content/drive/MyDrive/output/model_final.pth"
cfg = get_cfg()
cfg.merge_from_file(cfg_file)
cfg.MODEL.WEIGHTS = weight_file
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
predictor = DefaultPredictor(cfg)

# Function to process image and get prediction


# Function to plot the processed image and prediction
def plot_image(image):

    outputs = predictor(image)
    v = Visualizer(image[:, :, ::-1], metadata=None, scale=0.6)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plotted_image = plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))

    return plotted_image

# Flask app
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("index.html")



@app.route("/upload",methods=["POST"])
def upload():
    if  request.method == "POST":
        uploaded_file = request.files["image"]
        image = Image.open(uploaded_file)
        img = np.array(image)
        #image = Image.open(uploaded_file)
         # Perform prediction using the model

        plotted_image = plot_image(img)


        # Save the plotted image to a temporary file
        temp_file = "/tmp/image.png"
        plotted_image.figure.savefig(temp_file)
        plt.close()

        # Convert the result image to base64
        with open(temp_file, "rb") as f:
            result_image_base64 = base64.b64encode(f.read()).decode("utf-8")

        return render_template("index.html", result_image=result_image_base64)

    return render_template("index.html")

if __name__ == "__main__":

    try:
      app.run()
    except Exception as e:
      print("Error:", e)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://eaba-34-141-148-250.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [29/Jul/2023 03:50:44] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2023 03:50:45] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
INFO:werkzeug:127.0.0.1 - - [29/Jul/2023 03:50:55] "POST /upload HTTP/1.1" 200 -
